In [23]:
import os

In [24]:
%pwd

'd:\\Brain_MRI_Poject\\brain_classification'

In [25]:
os.chdir("../")

In [26]:
%pwd

'd:\\Brain_MRI_Poject'

In [27]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [28]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import yaml
from box import ConfigBox

In [29]:
path_to_yaml = "D:/Classification-Project/config/config.yaml"

with open(path_to_yaml) as yaml_file:
    content = yaml.safe_load(yaml_file)
    con = ConfigBox(content)
 

In [30]:
content

{'artifacts_root': 'artifacts',
 'data_ingestion': {'root_dir': 'artifacts/data_ingestion',
  'source_URL': 'https://github.com/Prateek07-Kumar/Brain-in-MRI-Images-for-Brain-Tumor-Detection/raw/main/brain_tumor_dataset.zip',
  'local_data_file': 'artifacts/data_ingestion/data.zip',
  'unzip_dir': 'artifacts/data_ingestion'},
 'prepare_base_model': {'root_dir': 'artifacts/prepare_base_model',
  'base_model_path': 'artifacts/prepare_base_model/base_model.h5',
  'updated_base_model_path': 'artifacts/prepare_base_model/base_model_updated.h5'},
 'prepare_callbacks': {'root_dir': 'artifacts/prepare_callbacks',
  'tensorboard_root_log_dir': 'artifacts/prepare_callbacks/tensorboard_log_dir',
  'checkpoint_model_filepath': 'artifacts/prepare_callbacks/checkpoint_dir/model.h5'},
 'training': {'root_dir': 'artifacts/training',
  'trained_model_path': 'artifacts/training/model.h5'}}

In [31]:
con

ConfigBox({'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/Prateek07-Kumar/Brain-in-MRI-Images-for-Brain-Tumor-Detection/raw/main/brain_tumor_dataset.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'prepare_base_model': {'root_dir': 'artifacts/prepare_base_model', 'base_model_path': 'artifacts/prepare_base_model/base_model.h5', 'updated_base_model_path': 'artifacts/prepare_base_model/base_model_updated.h5'}, 'prepare_callbacks': {'root_dir': 'artifacts/prepare_callbacks', 'tensorboard_root_log_dir': 'artifacts/prepare_callbacks/tensorboard_log_dir', 'checkpoint_model_filepath': 'artifacts/prepare_callbacks/checkpoint_dir/model.h5'}, 'training': {'root_dir': 'artifacts/training', 'trained_model_path': 'artifacts/training/model.h5'}})

In [32]:
def read_yaml(path_to_yaml: Path) -> ConfigBox:
    """reads yaml file and returns

    Args:
        path_to_yaml (str): path like input

    Raises:
        ValueError: if yaml file is empty
        e: empty file

    Returns:
        ConfigBox: ConfigBox type
    """
    try:
        with open(path_to_yaml) as yaml_file:
            content = yaml.safe_load(yaml_file)

            return ConfigBox(content)
    except BoxValueError:
        raise ValueError("yaml file is empty")
    except Exception as e:
        raise e

In [33]:
a = read_yaml(r"D:\Brain_MRI_Poject\brain_classification\params.yaml")

In [34]:
a

ConfigBox({'AUGMENTATION': True, 'IMAGE_SIZE': [224, 224, 3], 'BATCH_SIZE': 16, 'INCLUDE_TOP': False, 'EPOCHS': 1, 'CLASSES': 2, 'WEIGHTS': 'imagenet', 'LEARNING_RATE': 0.01})

In [2]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [50]:
import os
os.getcwd()
os.chdir(r"D:\Brain_MRI_Poject\brain_classification")

In [51]:
os.getcwd()

'D:\\Brain_MRI_Poject\\brain_classification'

In [52]:
read_yaml(PARAMS_FILE_PATH)

[2023-09-26 12:14:38,326: INFO: common: yaml file: params.yaml loaded successfully]


ConfigBox({'AUGMENTATION': True, 'IMAGE_SIZE': [224, 224, 3], 'BATCH_SIZE': 16, 'INCLUDE_TOP': False, 'EPOCHS': 1, 'CLASSES': 2, 'WEIGHTS': 'imagenet', 'LEARNING_RATE': 0.01})

In [53]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
    

In [54]:
CONFIG_FILE_PATH = Path("config/config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")

In [55]:
from box.exceptions import BoxValueError
cont = ConfigurationManager()

[2023-09-26 12:14:50,603: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-09-26 12:14:50,614: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-26 12:14:50,618: INFO: common: created directory at: artifacts]


In [56]:
CONFIG_FILE_PATH = Path("config/config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")

In [59]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [60]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [61]:
try:
    config = ConfigurationManager(CONFIG_FILE_PATH,PARAMS_FILE_PATH)
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-09-26 12:43:28,314: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-09-26 12:43:28,324: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-26 12:43:28,330: INFO: common: created directory at: artifacts]
[2023-09-26 12:43:28,335: INFO: common: created directory at: artifacts/data_ingestion]
[2023-09-26 12:43:28,339: INFO: 222777804: File already exists of size: ~ 7725 KB]


: 